In [4]:
from getpass import getpass
import urllib
import os
from random import sample
from glob import glob
import tensorflow as tf

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

In [5]:
#pred_path = "../../../project/masks/sresnet/marco_best"
pred_path = "../../../project/test_images/test_images"

pred_list = os.listdir(pred_path)
pred_list.sort()

print(pred_list)

['test_10.png', 'test_105.png', 'test_106.png', 'test_107.png', 'test_108.png', 'test_11.png', 'test_115.png', 'test_116.png', 'test_12.png', 'test_121.png', 'test_122.png', 'test_123.png', 'test_124.png', 'test_128.png', 'test_129.png', 'test_130.png', 'test_131.png', 'test_136.png', 'test_137.png', 'test_138.png', 'test_139.png', 'test_14.png', 'test_140.png', 'test_142.png', 'test_143.png', 'test_144.png', 'test_145.png', 'test_15.png', 'test_151.png', 'test_152.png', 'test_153.png', 'test_154.png', 'test_155.png', 'test_157.png', 'test_159.png', 'test_161.png', 'test_162.png', 'test_168.png', 'test_169.png', 'test_170.png', 'test_174.png', 'test_175.png', 'test_176.png', 'test_177.png', 'test_186.png', 'test_187.png', 'test_189.png', 'test_190.png', 'test_191.png', 'test_192.png', 'test_196.png', 'test_200.png', 'test_201.png', 'test_202.png', 'test_204.png', 'test_205.png', 'test_206.png', 'test_207.png', 'test_208.png', 'test_21.png', 'test_211.png', 'test_215.png', 'test_216.png

In [ ]:
"""
model_path = '../../../project/models/model_50epochs_2020_06_13_09_15_11.dms'
print(model_path)
model = tf.keras.models.load_model(model_path)
"""



In [9]:
#for image_path in pred_list:
#  print(image_path)

plt.figure(figsize=(18, 18))

image_path = pred_list[0]

IMG_RESIZE=416
#normalize=True

image = np.array(Image.open(os.path.join(pred_path, image_path)).resize((IMG_RESIZE, IMG_RESIZE)))

IMAGE_SHAPE = np.shape(image)
print(IMAGE_SHAPE)


(416, 416, 3)


<Figure size 1296x1296 with 0 Axes>

In [10]:
prob_map = image/255.0
#prob_map = prob_map.reshape(IMAGE_SHAPE)


In [11]:
image = np.expand_dims(image, 0)

print(np.shape(image))


(1, 416, 416, 3)


In [ ]:
#output = np.expand_dims(image, 0)
output = image.astype(np.uint8)
output = output.reshape((416,416,1))
output_img = tf.keras.preprocessing.image.array_to_img(output).resize((608, 608))

#prob_map = model.predict(image)[0]

#output = prob_map * 255.0

#output = output.astype(np.uint8)
#output_img = tf.keras.preprocessing.image.array_to_img(output).resize((608, 608))

print(np.shape(image))
#print(np.shape(img_parts))
#print(np.shape(output_img))

print(np.shape(output_img))
plt.imshow(output_img)
plt.show()

#print(output[0][0])
#print(output_img)
#print(np.shape(output))
#print(np.shape(output_img))

In [ ]:
prob_fb = np.tile(prob_map[np.newaxis,:,:],(2,1,1))
prob_fb[1,:,:] = 1 - prob_fb[0,:,:]


In [ ]:
prob_fg = np.expand_dims(prob_map,axis=0)
print(np.shape(prob_fg))

prob_bg = 1 - prob_fg
print(np.shape(prob_bg))

plt.figure(figsize=(15,5))
plt.subplot(1,2,1); plt.imshow(prob_fg[0]); plt.title('Foreground probability'); plt.axis('off'); plt.colorbar();
plt.subplot(1,2,2); plt.imshow(prob_bg[0]); plt.title('Background probability'); plt.axis('off'); plt.colorbar();


In [ ]:
import pydensecrf.densecrf as dcrf
from pydensecrf.utils import unary_from_softmax, create_pairwise_bilateral

# Inference without pair-wise terms
U = unary_from_softmax(prob_fb)  # note: num classes is first dim
print(np.shape(U))
d = dcrf.DenseCRF2D(416, 416, 2)
d.setUnaryEnergy(U)

# Run inference for 10 iterations
Q_unary = d.inference(10)

# The Q is now the approximate posterior, we can get a MAP estimate using argmax.
map_soln_unary = np.argmax(Q_unary, axis=0)

# Unfortunately, the DenseCRF flattens everything, so get it back into picture form.
map_soln_unary = map_soln_unary.reshape((416,416))

# And let's have a look.
plt.imshow(map_soln_unary); plt.axis('off'); plt.title('MAP Solution without pairwise terms');

In [ ]:
print(map_soln_unary)

In [ ]:
print(U)

In [ ]:
print(np.shape(image))
print(np.shape((output_img)))
print(image)
print(prob_map)
print(np.shape(prob_map))

print(image_path)


In [ ]:
"""
from scipy.stats import multivariate_normal

H, W, NLABELS = 400, 512, 2

# This creates a gaussian blob...
pos = np.stack(np.mgrid[0:H, 0:W], axis=2)
rv = multivariate_normal([H//2, W//2], (H//4)*(W//4))
probs = rv.pdf(pos)

import pydensecrf.densecrf as dcrf
from pydensecrf.utils import unary_from_softmax, create_pairwise_bilateral

probs = np.tile(probs[np.newaxis,:,:],(2,1,1))
probs[1,:,:] = 1 - probs[0,:,:]


In [ ]:
import pydensecrf.densecrf as dcrf
from pydensecrf.utils import unary_from_softmax, create_pairwise_bilateral

prob_map = np.tile(prob_map[np.newaxis,:,:],(2,1,1))
prob_map[1,:,:] = 1 - prob_map[0,:,:]

print(np.shape(prob_map))

In [ ]:
print(np.shape(pos))
print(np.shape(probs))

In [ ]:
plt.figure()
plt.hist(prob_map.flatten(),bins=50)
plt.show()

In [ ]:
U = unary_from_softmax(prob_map)
print(np.shape(prob_map))
print(np.shape(U))

In [ ]:
U = unary_from_softmax(probs)
pairwise_energy = create_pairwise_bilateral(sdims=(10,10), schan=(0.01,), img=img, chdim=2)

W = 608
H = 608
NLABELS = 2
d = dcrf.DenseCRF2D(W, H, NLABELS)
d.setUnaryEnergy(U)
d.addPairwiseEnergy(pairwise_energy, compat=10)  # `compat` is the "strength" of this potential.


In [ ]:
plt.figure(figsize=(18, 18))
plt.imshow(tf.keras.preprocessing.image.array_to_img(output_img))
plt.axis('off')

plt.show()